In [1]:
import os
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.losses import BinaryCrossentropy

import seaborn as sns
import helpers
import dataset
import test_lstm
import matplotlib.pyplot as plt

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

2022-11-22 03:39:44.657463: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
lie_trial_path = './processed_lie/' #60 entries
truth_trial_path = './processed_truth/' #61 entries

# MU3D_path = '/Users/frank/Downloads/dataSets/MU3D-dataset/processed/' # 320 entries

In [3]:
X, Y = dataset.path_preprocessing(truth_trial_path, lie_trial_path)

In [4]:
TEST_RATIO = 0.2

xTrain, xTest = train_test_split(X, test_size=TEST_RATIO, shuffle=False)
yTrain, yTest = train_test_split(Y, test_size=TEST_RATIO, shuffle=False)

In [5]:
LSTM_NEURONS = 32
DROPOUT = 0
EPOCHS = 10
OPTIM = 'adam'
# LOSS = 'binary_crossentropy'
LOSS = BinaryCrossentropy(from_logits=True)

n_inputRows = xTrain.shape[0] 
n_timesteps = xTrain.shape[1]
n_features = xTrain.shape[2]

def LSTM_Model(neuro, drop, loss, optim, step, feat):
  model = Sequential()
  model.add(LSTM(neuro, return_sequences=False, batch_input_shape=(None, step, feat)))

  # model.add(Dropout(drop))
  model.add(Dense(1, activation='linear')) #softmax

  #model.add(Lambda(lambda x: tf.cast(tf.argmax(x), tf.float32)))
  #model.add(Dense(1, activation=activ))

  model.compile(loss=loss, optimizer=optim, metrics=['accuracy'])
  return model

MODEL = LSTM_Model(LSTM_NEURONS, DROPOUT, LOSS, OPTIM, n_timesteps, n_features)
MODEL.summary()
MODEL.fit(xTrain, yTrain, validation_data=(xTest, yTest), epochs=EPOCHS, verbose=1)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 32)                6272      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 6,305
Trainable params: 6,305
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
1536/1536 [==============================] - 9s 5ms/step - loss: 0.0877 - accuracy: 0.9619 - val_loss: 0.0174 - val_accuracy: 0.9967
Epoch 2/10
1536/1536 [==============================] - 6s 4ms/step - loss: 0.0186 - accuracy: 0.9938 - val_loss: 0.0081 - val_accuracy: 0.9976
Epoch 3/10
1536/1536 [==============================] - 6s 4ms/step - loss: 0.0079 - accuracy: 0.9973 - val_loss: 0.0409 - val_accuracy: 0.9822
Epoch 4/10
1536/15

In [26]:
print(MODEL.predict(xTrain))

1536/1536 [==============================] - 2s 1ms/step
[[-13.105187 ]
 [ 12.9073515]
 [ 12.521594 ]
 ...
 [-12.206456 ]
 [-12.3230715]
 [ 13.146654 ]]


In [33]:
import importlib
importlib.reload(test_lstm)

lie_test_path = './test/Deceptive/processed/'

test_lstm.perdictSingleVideo(lie_test_path, MODEL)

24/24 [==============================] - 0s 1ms/step
[[4.091087 ]
 [4.091085 ]
 [4.0910954]
 [4.091095 ]
 [4.091136 ]
 [4.0911446]
 [4.091143 ]
 [4.09127  ]
 [4.0912805]
 [4.09157  ]
 [4.0915685]
 [4.0915666]
 [4.092217 ]
 [4.09223  ]
 [4.0933237]
 [4.0933814]
 [4.0933337]
 [4.095454 ]
 [4.0952287]
 [4.0971417]
 [4.0972953]
 [4.097242 ]
 [4.0993414]
 [4.0993285]
 [4.099756 ]
 [4.0999937]
 [4.0999804]
 [4.1002283]
 [4.1002035]
 [4.100263 ]
 [4.100189 ]
 [4.0997825]
 [4.0997624]
 [4.0997114]
 [4.100029 ]
 [4.099921 ]
 [4.100117 ]
 [4.10011  ]
 [4.100389 ]
 [4.100985 ]
 [4.101264 ]
 [4.1011853]
 [4.101238 ]
 [4.098672 ]
 [3.9350615]
 [3.9381256]
 [3.9412622]
 [4.0481033]
 [4.0469475]
 [4.0795527]
 [4.0782437]
 [4.0782757]
 [4.0870748]
 [4.086967 ]
 [4.0876923]
 [4.087844 ]
 [4.0882807]
 [4.078163 ]
 [4.077129 ]
 [4.007785 ]
 [4.0101347]
 [4.0096684]
 [3.9261642]
 [3.926528 ]
 [3.9072099]
 [3.9068995]
 [3.9067068]
 [3.9048462]
 [3.90445  ]
 [3.910727 ]
 [3.9113154]
 [3.91258  ]
 [3.9250493